# Part 3: Bringing Workflows to Production

For the purpose of this demo, let's add the `snowflake` package.

## Scheduling Python Jobs

In Snowflake, [Tasks](https://docs.snowflake.com/en/user-guide/tasks-intro) are used to automate and schedule jobs. Tasks can schedule the execution of a Notebook, a stored procedure, or arbitrary SQL statements. In this Notebook, you will learn how to use a Task to schedule a Python Stored Procedure to apply the Customer Loyalty Score UDAF on a schedule. Next, you will learn how to set up telemetry collection for the job and set up alerts.

## Creating a Python Stored Procedure

If you're not already familiar with [Stored Procedures](https://docs.snowflake.com/en/developer-guide/stored-procedure/stored-procedures-overview), they allow you to run *procedural* code, like loops, if/else blocks, and other patterns which are hard to do in SQL alone. 

To create a Python Stored Procedure, all we need is a Python function that accepts a Snowpark Session object as its first argument. In Snowflake we call this the "handler" for the procedure. For example, the Python function below is a bare-bones procedure handler that uses the Session object which returns an integer:

```python
from snowflake.snowpark import Session

def my_handler(sess: Session) -> int:
    n_rows = sess.table('my_table').count()
    return n_rows
```

Let's create a procedure to calculate the Customer Loyalty scores and write them to an output table. Then, we will schedule it with a Task so we can have up-to-date loyalty scores. 

In [ ]:
from snowflake.snowpark import Session
from datetime import datetime
from snowflake.snowpark.functions import lit
import logging

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
from snowflake.snowpark.types import IntegerType

# Handler for the procedure
def calc_loyalty_scores(sess: Session) -> int:
    logging.info('Starting loyalty score procedure')
    calculated_at = datetime.now()
    
    sess.sql("""
        SELECT
          CUSTOMER_ID,
          LoyaltyScoreUDAF(ORDER_TOTAL, ORDER_TS) AS loyalty_score
        FROM tasty_bytes_orders
        WHERE 
            ORDER_TOTAL IS NOT NULL 
            AND ORDER_TS IS NOT NULL
            AND CUSTOMER_ID IS NOT NULL
        GROUP BY CUSTOMER_ID
        ORDER BY loyalty_score DESC
    """).with_column("calculated_at", lit(calculated_at))\
    .write\
    .save_as_table('loyalty_scores', mode='append')

    # Returns the number of rows in the output table
    return sess.table('tasty_bytes_orders').count()


# Register the handler as a Stored Procedure
loyalty_sproc = session.sproc.register(
    calc_loyalty_scores,
    return_type=IntegerType(),
    name='calculate_loyalty_scores',
    is_permanent=True,
    stage_location='@my_stage/',
    replace=True
)

print(f'Created procedure "{loyalty_sproc.name}"')

In [ ]:
-- Call the stored proc for a test run
CALL CALCULATE_LOYALTY_SCORES();

Now let's create a Task to run this stored procedure on a schedule. Check out the docs for [`CREATE TASK`](https://docs.snowflake.com/en/sql-reference/sql/create-task) for more information, but at its core a Task needs a **schedule** and a **warehouse**. For example, the SQL below defines a Task that runs every hour with the Warehouse, `my_wh`, and calls a stored procedure named `my_stored_procedure`.

```sql
USE DATABASE TEST_DB;
USE SCHEMA TEST_SCHEMA;

CREATE TASK my_task
  WAREHOUSE = my_wh
  SCHEDULE = '60 MINUTES'
  AS
    CALL my_stored_procedure
```

And of course, you can define the same Task using Python!

```python
from datetime import timedelta
from snowflake.core.task import Cron, Task

tasks = root.databases["TEST_DB"].schemas["TEST_SCHEMA"].tasks

task = tasks.create(
    Task(
        name="my_task",
        definition="CALL my_stored_procedure",
        schedule=Cron("0 * * * *", "America/Los_Angeles"),
        warehouse="my_wh"
    ),
)
```

In [ ]:
# Let's create a Task in Python
from snowflake.core import Root
from snowflake.core.task import Cron, Task

root = Root(session)
tasks = root.databases["HOL_DB"].schemas["PUBLIC"].tasks
schedule = Cron("*/5 * * * *", "America/Los_Angeles")  # every 5 minutes

task = tasks.create(
    Task(
        name="loyalty_score",
        definition="CALL HOL_DB.PUBLIC.CALCULATE_LOYALTY_SCORES();",
        schedule=schedule,
        warehouse="my_wh"
    ),
    mode="orreplace"
)

# Start the task, will run every 5 minutes now
task.resume()

Now the Task, `loyalty_score`, should be running every 5 minutes. In the Database browser on the left, find your database and under the PUBLIC schema you should see the `loyalty_score` Task listed under the *Tasks* tab.

![Task Location](https://github.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/blob/main/images/task_location.png?raw=True) 

Click the task and it will bring you to the page for the Task. Click the **Run History** page to see the executions of the Task.

In [ ]:
# Now let's stop the Task's execution
task.suspend()

## Monitoring Python Workloads

Now we have a simple Python job scheduled to execute every 5 minutes. This raises the question: how do we monitor this job as it's running? How can we be notified if it fails overnight? How can we track performance over time? Snowflake provides a [suite of observability capabilities](https://www.snowflake.com/en/product/features/snowflake-trail/) to help you address those questions. Let's go over a few of Snowflake's observability capabilities:

1. Telemetry Collection: You can route any [logs](https://docs.snowflake.com/en/developer-guide/logging-tracing/logging), [metrics](https://docs.snowflake.com/en/developer-guide/logging-tracing/metrics), or [traces](https://docs.snowflake.com/en/developer-guide/logging-tracing/tracing) from your jobs to Snowflake's Event Tables for storage and alerting.
1. History tables: Use the [Query History](https://docs.snowflake.com/en/user-guide/ui-snowsight-activity), [Copy History](https://docs.snowflake.com/en/user-guide/data-load-monitor), and [Task History](https://docs.snowflake.com/en/user-guide/ui-snowsight-tasks) to monitor all usage in your account.
1. [Alerts and Notifications](https://docs.snowflake.com/en/developer-guide/builders/observability#label-observability-alerts-notifications): Alerts allow for customizable triggering conditions, actions, and a schedule, in combination with notification integrations for proactive monitoring.
1. [Extensibility with third-party tools](https://docs.snowflake.com/en/developer-guide/builders/observability#label-observability-tools-analysis-visualization): The Snowflake [event table](https://docs.snowflake.com/en/developer-guide/logging-tracing/event-table-setting-up) adopts [OpenTelemetry](https://opentelemetry.io/docs/) standards, so your Snowflake telemetry can easily be consumed by other ecosystem tools.

## Event Table Setup

The Event Table is the central storage system for logs, metrics, and traces in your account. A [default Event Table](https://docs.snowflake.com/developer-guide/logging-tracing/event-table-setting-up#use-the-default-event-table) is already created at `snowflake.telemetry.events`. Let's enable it:

In [ ]:
ALTER ACCOUNT SET EVENT_TABLE = snowflake.telemetry.events;

Next, we'll enable the collection of logs, metrics, and traces in our account.

In [ ]:
ALTER ACCOUNT SET LOG_LEVEL = 'INFO';
ALTER ACCOUNT SET METRIC_LEVEL = 'ALL';
ALTER ACCOUNT SET TRACE_LEVEL = 'ALWAYS';

We're now collecting logs, metrics, and traces for all jobs in our account. To test this, let's run our stored procedure in the cell below. After running this procedure, go to **Monitoring** > **Traces and Logs** > **Log Explorer**. You should see the log message "`Starting loyalty score procedure`" from the procedure in the **Log Explorer**. You can use the filters at the top to find the log message. Look under the Object column for the name of the procedure, "`CALCULATE_LOYALTY_SCORES()`".

![](https://raw.githubusercontent.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/refs/heads/main/images/log_explorer.png?token=GHSAT0AAAAAADCOV42CEFCF5UPR5AYAGZQ22AFO46A)

Click that log line and a pane will open on the right with more information about the log entry, like the object which emitted it, the warehouse used, and the file and line where the log line was emitted from. 

Click the button to go the Query associated with the log.

![](https://raw.githubusercontent.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/refs/heads/main/images/link_to_query_page.png?token=GHSAT0AAAAAADCOV42DHY4VTYASB2E6LYZU2AFPABQ)

TODO:
1. Show trace diagram
1. select span, show metrics

## Custom Telemetry



## Next Steps

Congratulations! Over these three Notebooks you've learned how to query data with Snowpark pandas, write user defined functions, external access, schedule jobs with Tasks, and monitor your jobs with logs, metrics, and traces. Now that you've completed this Hands-on Lab, here are some additional resources to keep learning.

### Create email notifications 

Try using what you learned about logs, metrics, and traces to create an alert if a job throws an error log or if a Task fails. This is a great way to stay on top of your scheduled data pipelines. To do so, use Snowflake's [alerts](http://docs.snowflake.com/en/user-guide/alerts) feature.

- [`SYSTEM$SEND_EMAIL()`](https://docs.snowflake.com/en/sql-reference/stored-procedures/system_send_email)
- [Send email notifications](https://docs.snowflake.com/en/user-guide/notifications/email-notifications)

### Visualize Data in Streamlit

If you're new to Python, you can keep learning by creating data visualizations and data-powered applications with Streamlit in Snowflake! This is a great way to learn more Python and incorporate it into your day-to-day workflows as a data professional.

- [Get Started with Snowpark and Streamlit](https://quickstarts.snowflake.com/guide/getting_started_with_snowpark_for_python_streamlit/#0)
- [Getting Started with Stramlit in Snowflake](https://docs.snowflake.com/en/developer-guide/streamlit/getting-started)
- [Streamlit in Snowflake](https://docs.snowflake.com/en/developer-guide/streamlit/about-streamlit)

